## 2024 ICG Final Project

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

def show_cv2_image(img):
   plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
   plt.show()

image_name = 'tomato.png'
image_path = './images/' + image_name

input_image = cv2.imread(image_path)

In [ ]:
from transformers import pipeline

pipe = pipeline(task="depth-estimation", model="LiheYoung/depth-anything-small-hf")
input_image_depth = Image.open(image_path)
depth_image = pipe(input_image_depth)["depth"]

plt.imshow(depth_image, cmap='gray')
plt.imsave('./depth'+'_'+image_name, depth_image,cmap='gray')

In [ ]:
import torch
from segment_anything import sam_model_registry
from segment_anything import SamAutomaticMaskGenerator
import supervision as sv

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"
sam = sam_model_registry[MODEL_TYPE](checkpoint='./weights/sam_vit_h.pth')
sam.to(device=DEVICE)
mask_generator = SamAutomaticMaskGenerator(sam)
image_bgr = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
result = mask_generator.generate(image_rgb)

sam_result_path = './masks_'+image_name[:-4]
os.makedirs(sam_result_path, exist_ok=True)

# exit()
for i in range(len(result)):
   mask  = result[i]['segmentation']
   result_mask = np.zeros_like(image_bgr)
   result_mask[mask==True] = 1
   cv2.imwrite(os.path.join(sam_result_path, 'mask_%d.png'%i), result_mask*255)


mask_annotator = sv.MaskAnnotator(color_lookup=sv.ColorLookup.INDEX)
detections = sv.Detections.from_sam(result)
annotated_image = mask_annotator.annotate(image_bgr, detections)

cv2.imwrite('./sam_'+image_name, annotated_image)

In [ ]:
mask_folder = os.listdir(sam_result_path)

### 計算深度並且產生 layers

In [ ]:
prior_list = []

for i in range(len(mask_folder)):
   mask = cv2.imread(os.path.join(sam_result_path, mask_folder[i]), 0)
   mask[mask!=0]=1
   pixel_num = mask.sum()
   prior = np.multiply(mask, depth_image)
   prior_list.append((mask_folder[i], prior.sum()/pixel_num))

other_mask = np.ones([input_image.shape[0], input_image.shape[1]])

for i in range(len(mask_folder)):
   mask = cv2.imread(os.path.join(sam_result_path, mask_folder[i]), 0)
   mask[mask!=0]=1
   other_mask[mask==1]=0
cv2.imwrite(os.path.join(sam_result_path, 'other_mask.png'), (other_mask*255).astype(np.uint8))
prior_list.append(('other_mask.png', -1*np.inf))

prior_list = sorted(prior_list, key=lambda mask: mask[1]) 

In [ ]:
crop_path = './crop_layers_'+image_name[:-4]
os.makedirs(crop_path, exist_ok=True)

In [ ]:
for idx in range(len(prior_list)):
   mask_name = prior_list[idx][0]
   mask_img = cv2.imread(os.path.join(sam_result_path, mask_name))
   mask_img[mask_img!=0]=1
   cv2.imwrite(os.path.join(crop_path, 'layer%s.png'%idx), mask_img*input_image)

# Start to paint the layer

In [ ]:
layer_folder = os.listdir(crop_path)
print(len(layer_folder))

In [ ]:
from paintly import paint

paint_path = './paint_layers_'+image_name[:-4]
os.makedirs(paint_path, exist_ok=True)

In [ ]:
for layer in layer_folder:
   img = cv2.imread(os.path.join(crop_path, layer))
   painting = paint(img, [2], T=5, curved=True, f_g=1)
   cv2.imwrite(os.path.join(paint_path, layer), (255*painting).astype(np.uint8))

In [ ]:
painted_folder = os.listdir(paint_path)
print(len(painted_folder))

In [ ]:
canvas = np.zeros_like(input_image)
print(canvas.shape)
print(paint_path, crop_path)

for idx in range(len(painted_folder)):
   layer_name = 'layer%s.png'%idx

   layer_img = cv2.imread(os.path.join(paint_path, layer_name))
   mask_img = cv2.imread(os.path.join(crop_path, layer_name))
   
   mask_img[mask_img!=0] = 1
   
   if idx==0:
      T = 20
   else:
      T = 70


   for yy in range(canvas.shape[0]):
      for xx in range(canvas.shape[1]):
         if layer_img[yy, xx, 0] > T or layer_img[yy, xx, 1] > T or layer_img[yy, xx, 2] > T:
            canvas[yy, xx, :] = layer_img[yy, xx, :] 

   # canvas[layer_img>T] = layer_img[layer_img>T].copy()

other_mask = cv2.imread(os.path.join(crop_path, 'layer0.png'))
canvas[other_mask>0] = other_mask[other_mask>0]
show_cv2_image(canvas)

In [ ]:
cv2.imwrite('./canvas_hs.png',canvas)